In [1]:
import pandas as pd
import json
import numpy as np
import folium
import nltk
import time
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import math
from geopy.geocoders import Nominatim
import re
from mpl_toolkits.basemap import Basemap


In [2]:
#df = pd.read_csv('tweets_en_april.csv',  header= -1)

df = pd.read_csv('processed/sentiments/twitter_sentiments_combined.csv')
#df.columns = ["City", "Sentiment", "Count"]
df.head()



,City,Sentiment,Count,Month,Language
0,Luzern,NEUTRAL,3069,april,de
1,Switzerland,POSITIVE,2246,april,de
2,Uster,POSITIVE,8,april,de
3,Winterthur,POSITIVE,83,april,de
4,Sursee,POSITIVE,3,april,de


In [3]:
switzerland = ['schweiz', 'schwyz', 'سويسرا', 'สวิตเซอร์แลนด์', 'スイス','thuy si','ch',
       'suica', 'suisse', 'suiza', 'suíça', 'suïssa', 'svajcarska',
       'sveitsi', 'svizzera', 'swiss', 'switzerland',
       'zwitserland', 'isviçre', 'Швейцария']

In [4]:
df['City'] = df['City'].map(lambda x: re.sub(r'[^.áâçàéèa-zA-ZÖöÜüÄä]+', ' ', x))
df['City'] = df['City'].map(lambda x: x.replace(". ", " ")) 
df['City'] = df['City'].map(lambda x: x.split(' ', 1)[0])
df['City'] = df['City'].map(lambda x: x.lower())
df = df.loc[~df['City'].isin(switzerland)]
df = df[pd.notnull(df['City'])]
df = df[['City', 'Sentiment', 'Count']]
df

,City,Sentiment,Count
0,luzern,NEUTRAL,3069
2,uster,POSITIVE,8
3,winterthur,POSITIVE,83
4,sursee,POSITIVE,3
5,saanen,NEUTRAL,74
6,baden,NEUTRAL,1398
7,meilen,NEUTRAL,8
9,waldenburg,NEUTRAL,659
10,schleitheim,NEUTRAL,2
11,willisau,NEUTRAL,17


In [ ]:
trial = df.ix[:5]
trial


In [5]:
df = df.loc[df['Sentiment'].isin(['POSITIVE','NEGATIVE'])]
df.head()





,City,Sentiment,Count
2,uster,POSITIVE,8
3,winterthur,POSITIVE,83
4,sursee,POSITIVE,3
15,biel,POSITIVE,5
17,sarnen,POSITIVE,1


In [6]:
df['City'] = pd.core.strings.str_strip(df['City'])
df.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,City,Sentiment,Count
2,uster,POSITIVE,8
3,winterthur,POSITIVE,83
4,sursee,POSITIVE,3
15,biel,POSITIVE,5
17,sarnen,POSITIVE,1


In [7]:
df = df.groupby(['City','Sentiment'])[('Count')].sum()
df

City              Sentiment
                  NEGATIVE      3032
                  POSITIVE      7086
.                 NEGATIVE         1
                  POSITIVE         1
..                NEGATIVE         3
                  POSITIVE         2
...anti           POSITIVE         1
.ch               NEGATIVE         3
                  POSITIVE        14
.fr               NEGATIVE        20
                  POSITIVE        78
a                 NEGATIVE         9
                  POSITIVE        36
aachen            POSITIVE         3
aadorf            NEGATIVE         2
                  POSITIVE        47
aarau             NEGATIVE       389
                  POSITIVE      1164
aarberg           NEGATIVE         1
                  POSITIVE         4
aarburg           NEGATIVE        53
                  POSITIVE        84
aargau            NEGATIVE       978
                  POSITIVE      1231
aarhus            POSITIVE         1
abadons           NEGATIVE         2
          

In [8]:
df = df.reset_index()
df

,City,Sentiment,Count
0,,NEGATIVE,3032
1,,POSITIVE,7086
2,.,NEGATIVE,1
3,.,POSITIVE,1
4,..,NEGATIVE,3
5,..,POSITIVE,2
6,...anti,POSITIVE,1
7,.ch,NEGATIVE,3
8,.ch,POSITIVE,14
9,.fr,NEGATIVE,20


In [9]:
a = df.loc[df.groupby("City")["Count"].idxmax()]
a.head(20)
a

,City,Sentiment,Count
1,,POSITIVE,7086
2,.,NEGATIVE,1
4,..,NEGATIVE,3
6,...anti,POSITIVE,1
8,.ch,POSITIVE,14
10,.fr,POSITIVE,78
12,a,POSITIVE,36
13,aachen,POSITIVE,3
15,aadorf,POSITIVE,47
17,aarau,POSITIVE,1164


In [10]:
geolocator = Nominatim()

def getCoordinates (row):
    #time.sleep(0.01)
    print(row['City'])
    try:
        address, (x, y) = geolocator.geocode(row['City'])
        address = address.split(",")
        country = address[-3]
        return x, y, country
    except Exception:
        return np.nan, np.nan, " "
     


### The above algorithm runs very long for a large dataframe!
The solution is in file: coordinatesAndCountriesGivenSentiments10m.csv

In [16]:
#a["Latitude"], a["Longitude"], a["Country"]= zip(*a.apply (lambda row: getCoordinates (row),axis=1))
#a

In [12]:
#outFile = "coordinatesForGivenSentiments10m.csv"
#a.to_csv(outFile,index=None)

In [43]:
a = pd.read_csv('coordinatesForGivenSentiments10m.csv')
a['Country'].dtype
a.head()

,City,Sentiment,Count,Latitude,Longitude,Country
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse
1,aadorf,POSITIVE,47,47.491578,8.902953,Suisse
2,aarau,POSITIVE,1164,47.392715,8.044445,Suisse
3,aarberg,POSITIVE,4,47.044335,7.275300,Suisse
4,aarburg,POSITIVE,84,47.320642,7.899360,Suisse


### Find only the data for Switzerland

In [44]:
a['Country'] = pd.core.strings.str_strip(a['Country'])
a['City'] = pd.core.strings.str_strip(a['City'])
allSuisse = a.loc[a['Country'] == 'Suisse']
allSuisse

,City,Sentiment,Count,Latitude,Longitude,Country
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse
1,aadorf,POSITIVE,47,47.491578,8.902953,Suisse
2,aarau,POSITIVE,1164,47.392715,8.044445,Suisse
3,aarberg,POSITIVE,4,47.044335,7.275300,Suisse
4,aarburg,POSITIVE,84,47.320642,7.899360,Suisse
5,aargau,POSITIVE,1231,47.412396,8.194832,Suisse
6,adelboden,POSITIVE,60,46.492721,7.558762,Suisse
7,adligenswil,POSITIVE,10,47.070535,8.368244,Suisse
8,adliswil,POSITIVE,33,47.311762,8.524910,Suisse
9,aegerten,NEGATIVE,2,47.120241,7.288497,Suisse


In [20]:
#outFile = "swissTwitterSentimentsAndCoordinates10m.csv"
#allSuisse.to_csv(outFile,index=None)

In [45]:
allSuisse = pd.read_csv('swissTwitterSentimentsAndCoordinates10m.csv')
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse
1,aadorf,POSITIVE,47,47.491578,8.902953,Suisse
2,aarau,POSITIVE,1164,47.392715,8.044445,Suisse
3,aarberg,POSITIVE,4,47.044335,7.275300,Suisse
4,aarburg,POSITIVE,84,47.320642,7.899360,Suisse


In [46]:
allSuisse['City']= allSuisse['City'].map(lambda x: x.lower())
#allSuisse = allSuisse.loc[allSuisse.groupby(['City','Sentiment'])["Count"].sum()]
#allSuisse = allSuisse.groupby(['City','Sentiment'])["Count"].sum()
#allSuisse = allSuisse.dropna() 
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse
1,aadorf,POSITIVE,47,47.491578,8.902953,Suisse
2,aarau,POSITIVE,1164,47.392715,8.044445,Suisse
3,aarberg,POSITIVE,4,47.044335,7.275300,Suisse
4,aarburg,POSITIVE,84,47.320642,7.899360,Suisse


In [ ]:
#allSuisse = allSuisse.groupby(['City','Sentiment','Latitude','Longitude'])["Count"].sum()

#allSuisse

In [ ]:
#allSuisse = allSuisse.reset_index()
#allSuisse

In [47]:
def getColour (row):
   if row['Sentiment'] == 'NEUTRAL' :
      return 'blue'
   if row['Sentiment'] == 'POSITIVE' :
      return 'green'
   if row['Sentiment'] == 'NEGATIVE':
      return 'red'
   return 'black'

In [48]:
allSuisse['Colour'] = allSuisse.apply (lambda row: getColour (row),axis=1)
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country,Colour
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse,green
1,aadorf,POSITIVE,47,47.491578,8.902953,Suisse,green
2,aarau,POSITIVE,1164,47.392715,8.044445,Suisse,green
3,aarberg,POSITIVE,4,47.044335,7.275300,Suisse,green
4,aarburg,POSITIVE,84,47.320642,7.899360,Suisse,green


In [49]:
cantons_geo = r'ch-cantons.topojson.json'
canton_map = folium.Map(location=[46.8, 8.33],tiles='OpenStreetMap', zoom_start=8)
canton_map.choropleth(geo_path = cantons_geo, topojson='objects.cantons', fill_color='#3186cc')
canton_map

In [26]:
# Get the location of each city and plot it

for index, row in allSuisse.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row["City"],icon=folium.Icon(color=row["Colour"], icon='ok-sign')).add_to(canton_map)

#canton_map.save( 'mymap.html')


In [27]:
canton_map

In [ ]:
# kanton_map.plot(x,y,marker='o',color='Red',markersize=int(math.sqrt(count))*scale)

In [28]:
canton_map.save( 'mapForTwitterGivenSentiments10months.html')

In [30]:
for index, row in allSuisse.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], radius=row["Count"],popup=row["City"],color=row["Colour"],fill_color=row["Colour"]).add_to(canton_map)


In [31]:
canton_map

In [ ]:
#canton_map.save( 'mapForTwitterGivenSentiments10monthsRadius.html')

In [50]:
allSuisse['Count/10'] = allSuisse['Count'].map(lambda x: x/10)
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country,Colour,Count/10
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse,green,1.4
1,aadorf,POSITIVE,47,47.491578,8.902953,Suisse,green,4.7
2,aarau,POSITIVE,1164,47.392715,8.044445,Suisse,green,116.4
3,aarberg,POSITIVE,4,47.044335,7.275300,Suisse,green,0.4
4,aarburg,POSITIVE,84,47.320642,7.899360,Suisse,green,8.4


In [51]:
for index, row in allSuisse.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], radius=row["Count/10"],popup=row["City"],color=row["Colour"],fill_color=row["Colour"]).add_to(canton_map)


In [52]:
canton_map

In [53]:
canton_map.save( 'mapForTwitterGivenSentiments10monthsRadius.html')